In [185]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import plotly.express as px

In [186]:
transfers = pd.read_csv(Path.cwd()/'prepped/final.csv')
inflation = pd.read_csv(Path.cwd()/'prepped/inflation.csv')
age_coef = pd.read_csv(Path.cwd()/'prepped/age_coeffiecient.csv')

/var/folders/jm/4ddznsn94j7_pznhvk_1rwlw0000gn/T/ipykernel_94520/599693918.py:1: DtypeWarning: Columns (38,59,75,88,92,103,149,170,186,199,203,214,260,281,297,310,314,325,371,392,408,421,425,436,704,725,741,754,758,769,815,836,852,865,869,880,926,947,963,976,980,991,1037,1058,1074,1087,1091,1102) have mixed types. Specify dtype option on import or set low_memory=False.
  transfers = pd.read_csv(Path.cwd()/'prepped/final.csv')


$EFF_{t+1} = \frac{a*(MARKETVAL_{t+1} - MARKETVAL_{t}) + (FEE_{t} - MARKETVAL_{t})}{2* MARKETVAL_{t} * (1+ \pi_{t+1})}$

In [187]:
def effectiveness(row):
    return ((row['age_coef'] * (1 + row['inflation']) ** -1 * row['marketval_0'] - row['marketval_-1'] + (row['marketval_-1'] - row['fee']))) / (2 * row['marketval_-1'])

In [188]:
transfers = transfers[['name', 'age', 'season', 'club_from', 'club_to','fee', 'marketval_-1', 'marketval_0']].dropna()

In [189]:
inflation.inflation = inflation.inflation.shift(-1)
transfers.fee *= 10**6
transfers = transfers[transfers.fee > 0]

In [190]:
transfers = transfers.merge(age_coef, how='left', on='age').merge(inflation, how='left', on='season')

In [191]:
transfers['eff'] = transfers.apply(effectiveness, axis=1)

In [192]:
transfers.sort_values('eff', ascending=False).head(50)

,name,age,season,club_from,club_to,fee,marketval_-1,marketval_0,age_coef,inflation,eff
120,Lucas Pérez,30,19/20,West Ham United,Deportivo Alavés,2300000.0,4000000.0,12000000.0,1.116962,-0.078351,1.530375
198,E. Martínez,28,20/21,Arsenal,Aston Villa,17400000.0,8000000.0,35000000.0,0.912789,-0.233083,1.516075
66,Diego Carlos,26,19/20,Nantes,Sevilla,15000000.0,13000000.0,50000000.0,0.787918,-0.078351,1.067115
124,F. Niederlechner,28,19/20,Freiburg,Augsburg,2500000.0,3500000.0,10000000.0,0.912789,-0.078351,1.057696
493,I. Provedel,28,22/23,Spezia,Lazio,2550000.0,3500000.0,12000000.0,0.912789,0.194958,0.945200
250,J. El Yamiq,28,20/21,Genoa,Real Valladolid,250000.0,650000.0,1200000.0,0.912789,-0.233083,0.906344
235,A. Younes,27,20/21,Napoli,Eintracht Frankfurt,750000.0,6000000.0,10000000.0,0.866515,-0.233083,0.879057
371,Reinildo Mandava,28,21/22,Lille,Atlético Madrid,3000000.0,9000000.0,22000000.0,0.912789,0.199843,0.763148
185,André Silva,24,20/21,Milan,Eintracht Frankfurt,3000000.0,24000000.0,45000000.0,0.670855,-0.233083,0.757572
265,C. Richards,20,20/21,Bayern München,Hoffenheim,250000.0,2200000.0,7500000.0,0.362848,-0.233083,0.749645


In [193]:
transfers[transfers.fee > 1000].sort_values('eff', ascending=True).head(50)

,name,age,season,club_from,club_to,fee,marketval_-1,marketval_0,age_coef,inflation,eff
163,A. Masiello,33,19/20,Atalanta,Genoa,100000000.0,1000000.0,1200000.0,1.397068,-0.078351,-49.090499
92,Bruno Jordão,20,19/20,Lazio,Wolverhampton Wanderers,8900000.0,1000000.0,2300000.0,0.362848,-0.078351,-3.997252
502,M. Tel,17,22/23,Rennes,Bayern München,20000000.0,2000000.0,20000000.0,0.273417,0.194958,-3.855957
521,N. Ahamada,20,22/23,Stuttgart,Crystal Palace,12000000.0,1500000.0,10000000.0,0.362848,0.194958,-2.987837
244,A. Tripaldelli,21,20/21,Sassuolo,Cagliari,3000000.0,450000.0,1000000.0,0.451151,-0.233083,-2.679706
60,Pedro Neto,19,19/20,Lazio,Wolverhampton Wanderers,17900000.0,2500000.0,15000000.0,0.326293,-0.078351,-2.517904
178,W. Fofana,19,20/21,Saint-Étienne,Leicester City,35000000.0,5400000.0,40000000.0,0.326293,-0.233083,-1.664959
531,J. Kiwior,22,22/23,Spezia,Arsenal,19500000.0,3000000.0,25000000.0,0.531726,0.194958,-1.395941
13,Rafael Leão,20,19/20,Lille,Milan,49500000.0,15000000.0,20000000.0,0.362848,-0.078351,-1.387537
360,C. Wood,30,21/22,Burnley,Newcastle United,30000000.0,8000000.0,10000000.0,1.116962,0.199843,-1.293173
